# Fourier transformations

## Initializing

In [ ]:
import scipy as sp
import pylab as pl
import cmath as cm

def source_function(x):
    return x #sp.sin(3*x) * x - x*x

source_function = lambda x: sp.sin(2 * sp.pi * 2 * x) #(lambda x: sp.sin(6*x) + sp.cos(5*x))
noise_function = lambda x: 0.1 * sp.sin(2 * sp.pi * 15 * x) + 0.05 * sp.sin(2 * sp.pi * 10 * x) + 0.05 * sp.cos(2 * sp.pi * 20 * x)
result_function = lambda x: source_function(x) + noise_function(x)
source_function_period = 2.0
N = 200
#source_x = sp.arange(0, source_function_period, (source_function_period/N))
result_x = sp.arange(0, N, 1)

source_x = sp.arange(0, source_function_period, (source_function_period/N))
source_y = sp.vectorize(source_function)(source_x)
noise_y = sp.vectorize(result_function)(source_x)

pl.figure(figsize = (15 , 5))

pl.subplot(1, 2, 1)
pl.plot(source_x, source_y)
pl.title("Source function")
pl.grid()

pl.subplot(1, 2, 2)
pl.plot(source_x, noise_y)
pl.title("Noised function")
pl.grid()
pl.show()


## Filter characteristics

In [ ]:
filter_power = N/(2*(int)(source_function_period))

def reverse_transformation(data):
    result = []
    n = len(data)
    for i in range(n):
        result.append(0)
        for j in range(n):
            result[i] += data[j] * sp.cos(2 * sp.pi * i * (-j)/float(n))
    return sp.roll([x/n for x in result], n/2)

def transient_characteristic(x, bandwith = 5, power = filter_power):
    if (x < bandwith or (x > power-bandwith and x < power)):
    #if (x < bandwith):
        return 1
    else:
        return 0

fourier_x = sp.arange(0, filter_power, 1)
transient_y = sp.vectorize(transient_characteristic)(fourier_x)

impulse_y = reverse_transformation(transient_y)

# noise_y = sp.vectorize(result_function)(source_x)

pl.figure(figsize = (15 , 5))

pl.subplot(1, 2, 1)
pl.plot(fourier_x, transient_y)
pl.title("Transient characteristic")
pl.grid()

pl.subplot(1, 2, 2)
pl.plot(fourier_x, impulse_y)
pl.title("Impulse characteristic")
pl.grid()

pl.show()

In [ ]:
def discrete_transformation(data):
    n = len(data)
    discrete_result = [0] * n
    for i in range(n):
        for j in range(n):
            discrete_result[i] += complex(
                sp.cos(2 * sp.pi * i * (j/float(n))), 
                sp.sin(2 * sp.pi * i * (j/float(n)))
            ) * data[j]
    return [abs(x) for x  in discrete_result]

source_fourier_x = sp.arange(0, N, 1)
source_reverse_y = map(lambda x: x/(N/2), discrete_transformation(noise_y))

impulse_discrete_reversed = discrete_transformation(impulse_y)

pl.figure(figsize = (15 , 5))

pl.subplot(1, 2, 1)
pl.plot(source_fourier_x, source_reverse_y)
pl.title("Source function transformed")
pl.grid()

pl.subplot(1, 2, 2)
pl.plot(fourier_x, impulse_discrete_reversed)
pl.title("Filtering result")
pl.grid()
pl.show()

## Filtering

In [ ]:
def convolution(f, g, N, M):
    con = [0] * N
    for i in range(0, (int)(N)):
        for j in range(min(i, M)):
#             print("i: %d, j: %d" % (i, j))
            f_x = (i-j)
            con[i] += f[f_x] * g[j]
    return [x for x in con]

filtering_result = convolution(noise_y, impulse_y, N, filter_power)

pl.figure(figsize = (15 , 5))

pl.subplot(1, 2, 1)
pl.plot(source_x, noise_y)
pl.title("Source function")
pl.grid()

pl.subplot(1, 2, 2)
pl.plot(source_x, filtering_result)
pl.title("Filtering result")
pl.grid()
pl.show()